In [8]:
import os
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests
from langchain_core.tools import InjectedToolArg
from langchain_community.tools import DuckDuckGoSearchRun
from typing import Annotated
import json
from langchain.agents import AgentExecutor, create_react_agent
#from langgraph.prebuilt import create_react_agent
from langchain import hub

In [15]:
search_tool=DuckDuckGoSearchRun()

In [23]:
@tool
def get_weather(city: str) -> str:

    """This is a function that gets the weather information of a given city"""
    api_key = 'bd23a21fe5f3708234a29e76d154d027'  # Replace with your OpenWeatherMap API key
    base_url = 'http://api.openweathermap.org/data/2.5/weather'

    params = {
        'q': city,
        'appid': api_key,
        'units': 'metric'  # Use 'imperial' for Fahrenheit
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        weather_details = {
            'city': data['name'],
            'temperature': data['main']['temp'],
            'weather': data['weather'][0]['description'],
            'humidity': data['main']['humidity'],
            'wind_speed': data['wind']['speed'],
        }
        return weather_details
    else:
        return {'error': 'City not found or API limit reached'}

# Example usage:
# print(get_weather('London'))

In [24]:
llm=ChatOllama(model='llama3.1:8b')

In [25]:
prompt=hub.pull('hwchase17/react')

In [26]:
agent=create_react_agent(
    llm=llm,
    tools=[search_tool,get_weather],
    prompt=prompt
)

In [27]:
agent_executor=AgentExecutor(
    agent=agent,
    tools=[search_tool,get_weather],
    verbose=True
)

In [28]:
response=agent_executor.invoke({"input":"find the capital of Andhra Pradesh and what is the current weather of that city?"})
print(response)



> Entering new AgentExecutor chain...
To find the capital of Andhra Pradesh, I need to look up some information. I'll search for it on DuckDuckGo.

Action: duckduckgo_search
2 The region is under the jurisdiction of Andhra Pradesh Capital Region Development Authority and covers an area of 8,352.69 km 2 (3,224.99 sq mi ... The capital of Andhra Pradesh is still anybody ’ s guess ... The new capital city for the bifurcated Andhra Pradesh still remains a mystery ... Amaravati, the proposed capital of Andhra Pradesh , was strategically chosen in 2014 following the bifurcation of the state. He highlighted that during the announcement of Amaravati as the state capital after Andhra Pradesh s bifurcation, public opinion was not adequately ... The new capital city of Andhra pradesh is proposed in Guntur District, north of Guntur City and will be developed under Capital Region Development ...Question: find the capital of Andhra Pradesh and what is the current weather of that city?
Thought: To 

In [29]:
response['output']

'The capital of Andhra Pradesh is Amaravati, and the current weather there is clear sky with a temperature of 17.01°C and humidity of 89%.'